In [ ]:
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from transformers import BertTokenizer
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
!gdown 1zASdz9hUY6NC6p7h9ef5trbv4B3uUpJr

In [ ]:
df = pd.read_csv('/content/bot_detection_data.csv')
df

O dataset contém 50.000 linhas e 11 colunas, sendo elas:

* User ID: ID do usuário (numérico).
* Username: Nome do usuário (texto).
* Tweet: O conteúdo do tweet (texto).
* Retweet Count: Contagem de retweets (numérico).
* Mention Count: Contagem de menções (numérico).
* Follower Count: Contagem de seguidores (numérico).
* erified: Indicação se o usuário é verificado (booleano).
* Bot Label: Rótulo que indica se é um bot (1) ou não (0) (numérico).
* Location: Localização do usuário (texto).
* Created At: Data de criação do tweet (texto).
* Hashtags: Hashtags utilizadas no tweet (texto).

In [ ]:
df.columns

In [ ]:
# Visualizando as primeiras informações dos dados
df.head()  # Primeiras linhas do DataFrame
df.info()  # Informações sobre os tipos de dados e valores nulos
df.describe()  # Estatísticas descritivas das colunas numéricas

In [ ]:
# Configurando estilo dos gráficos
plt.style.use('ggplot')

# Distribuição da variável "Bot Label"
plt.figure(figsize=(6,4))
sns.countplot(x='Bot Label', data=df, palette='Set2')
plt.title("Distribuição de Bots vs Não Bots")
plt.xlabel("Bot Label")
plt.ylabel("Contagem")
plt.show()

A proporção de bots (1) e não bots (0) está balanceada, com quantidades semelhantes em cada grupo.

In [ ]:
# Verificando a distribuição de variáveis numéricas principais com relação à variável alvo
plt.figure(figsize=(16, 6))

# Subplot para "Retweet Count"
plt.subplot(1, 3, 1)
sns.boxplot(x='Bot Label', y='Retweet Count', data=df, palette='Set3')
plt.title("Distribuição de Retweets por Bot Label")

# Subplot para "Mention Count"
plt.subplot(1, 3, 2)
sns.boxplot(x='Bot Label', y='Mention Count', data=df, palette='Set3')
plt.title("Distribuição de Menções por Bot Label")

# Subplot para "Follower Count"
plt.subplot(1, 3, 3)
sns.boxplot(x='Bot Label', y='Follower Count', data=df, palette='Set3')
plt.title("Distribuição de Seguidores por Bot Label")

plt.tight_layout()
plt.show()

**Distribuições de Retweets, Menções e Seguidores por Bot Label:**

* **Retweet Count:** Bots tendem a ter uma distribuição de retweets mais alta e com mais outliers, sugerindo que podem estar envolvidos em campanhas de retweet automáticas.
* **Mention Count:** A distribuição de menções é semelhante entre bots e não bots, mas com mais variabilidade para os bots.
* **Follower Count:** A contagem de seguidores também apresenta uma maior variabilidade entre bots, o que pode indicar a presença de bots com muitos ou poucos seguidores.

In [ ]:
# A coluna correta é "Hashtags" com a letra maiúscula
df['Hashtags'] = df['Hashtags'].fillna('None')

# Conversão da coluna "Created At" para datetime
df['created_at'] = pd.to_datetime(df['Created At'], errors='coerce')

# Verificar se existem valores nulos após as correções
missing_data = df.isnull().sum()

# Exibir o resultado do tratamento
missing_data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalizar as colunas numéricas: 'Retweet Count', 'Mention Count', 'Follower Count'
scaler = MinMaxScaler()

df[['Retweet Count', 'Mention Count', 'Follower Count']] = scaler.fit_transform(df[['Retweet Count', 'Mention Count', 'Follower Count']])

# Verificar a distribuição do "Bot Label" para o balanceamento
bot_label_distribution = df['Bot Label'].value_counts(normalize=True) * 100

# Exibir a distribuição da variável alvo
bot_label_distribution

## Preparação dos dados para o modelo BERT

In [ ]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Definir o nome do modelo pré-treinado
pretrained_model_name = 'distilbert-base-uncased'

# Carregar o tokenizador
tokenizer = DistilBertTokenizer.from_pretrained(pretrained_model_name)

# Carregar o modelo pré-treinado
bert_model = TFDistilBertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2)

# Pré-processamento dos dados
X = df[['Tweet']]  # Usando a coluna de texto 'Tweet'
y = df['Bot Label']  # Usando a coluna 'Bot Label' como target

# Dividindo o dataset para usar apenas 10% no treinamento
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, random_state=42)

# Tokenização usando o DistilBERT
def encode_tweets(texts, tokenizer, max_len=128):
    return tokenizer(
        texts.tolist(),
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

train_encodings = encode_tweets(X_train['Tweet'], tokenizer)
test_encodings = encode_tweets(X_test['Tweet'], tokenizer)

# Convertendo para tensores
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))

# Definir as entradas corretamente
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# Lambda layer para encapsular o modelo BERT, especificando o formato de saída explicitamente
bert_output = Lambda(lambda x: bert_model.distilbert(input_ids=x[0], attention_mask=x[1])[0],
                     output_shape=(128,768))([input_ids, attention_mask])

# Adicionando camadas extras ao modelo
dropout_layer = Dropout(0.3)(bert_output[:, 0, :])  # Usando apenas o primeiro token [CLS]
dense_layer = Dense(128, activation='relu')(dropout_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Criando o modelo final
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_layer)

# Compilar o modelo com otimizador e hiperparâmetros ajustados
optimizer = Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Resumo da arquitetura do modelo
model.summary()

In [ ]:
# Treinamento do modelo com Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Ajuste dos hiperparâmetros e treinamento
model.fit(
    train_dataset.batch(16),  # Certifique-se que o dataset está sendo carregado corretamente
    validation_data=test_dataset.batch(256),
    epochs=5,  # Aumentei para 5 épocas
    callbacks=[early_stopping]
)

In [ ]:
# Avaliação do modelo
loss, accuracy = model.evaluate(test_dataset.batch(16))
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

# Salvando o modelo treinado
model.save("/kaggle/working/bert_bot_detection_model.h5")

In [ ]:
# Avaliação do modelo
loss, accuracy = model.evaluate(test_dataset.batch(16))
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")